In [1]:
import pandas as pd
import os
import json
import glob
from shapely.geometry  import shape, Point
from geopy.geocoders import Nominatim
import numpy as np

In [2]:
output_name = '5_years'

In [3]:
geolocator = Nominatim(user_agent="test_tk")

In [4]:
os.makedirs('./results', exist_ok=True )

In [5]:
data_path = '../download_with_python/data/'
cur_dir = os.getcwd()

In [6]:
os.chdir(data_path)
files = glob.glob("*.json")
os.chdir(cur_dir)

In [8]:
crimes_dict = {}
offenders_dict = {}
env_dict = {}
crimes_i = 0
env_i = 0
offenders_i = 0

for file in files:
    name = file.split(sep="_")[0]
    if name == 'crime5':
        crimes_dict[name+f'_{crimes_i}'] = pd.read_json((data_path+file), lines=True)
        crimes_i += 1
    elif name == 'offenders':
        offenders_dict[name+f'_{offenders_i}'] = pd.read_json((data_path+file), lines=True)
        # drop registered as homeless offenders
        offenders_dict[name+f'_{offenders_i}'] = offenders_dict[name+f'_{offenders_i}'][~(offenders_dict[name+f'_{offenders_i}']['block'] == '(registered as homeless)')]
        # replace anonymized house number with 0 (replaces X with 0)
        offenders_dict[name+f'_{offenders_i}']['block'] = offenders_dict[name+f'_{offenders_i}']['block'].str.replace('X','0') + ' CHICAGO'
        offenders_dict[name+f'_{offenders_i}'][['stnum','restofadd']] = offenders_dict[name+f'_{offenders_i}']['block'].str.split(" ", n=1, expand=True)
        offenders_dict[name+f'_{offenders_i}']['stnum'] = offenders_dict[name+f'_{offenders_i}']['stnum'].astype(int) 
        offenders_dict[name+f'_{offenders_i}']['fixed_addr'] = offenders_dict[name+f'_{offenders_i}']['stnum'].astype(str) + ' ' + offenders_dict[name+f'_{offenders_i}']['restofadd']
    elif name == 'envcomplaint5':
        env_dict[name+f'_{env_i}'] = pd.read_json((data_path+file), lines=True)
        env_i += 1
    else:
        print(f'error on {file}')



NameError: name 'i' is not defined

In [ ]:
# TO DO
# make merger function to merge like for like

In [ ]:
# define geojsons
with open('../../geojsons/Boundaries - Community Areas (current).geojson') as f:
    commarea = json.load(f)

In [ ]:
def merge_df(dictionary):
    merged_df = pd.DataFrame()
    for datfram in dictionary:
        merged_df= pd.concat([merged_df,dictionary[datfram]],axis=0)
        print(merged_df.shape[0])
    return merged_df

In [ ]:
crimes_dict.keys()

dict_keys(['crime5_0', 'crime5_1', 'crime5_2'])

In [ ]:
merged_crimes = merge_df(crimes_dict)
merged_offenders = merge_df(offenders_dict)
merged_envcomplaints = merge_df(env_dict)

500000
1000000
1181139
2864
9410


In [ ]:
def get_commarea_env(dataframe):
    dataframe['community_area']=''
    for row in dataframe.index:
        point = Point(dataframe['location.coordinates'][row][0],dataframe['location.coordinates'][row][1])
        for feature in commarea['features']:
            polygon = shape(feature['geometry'])
            if polygon.contains(point):
                dataframe['community_area'][row] = feature['properties']['area_numbe']

In [ ]:
qq = geolocator.geocode('7200 S SOUTH SHORE DR CHICAGO')


In [ ]:
latlong = [qq.latitude, qq.longitude]
point = Point(latlong[1], latlong[0])
for feature in commarea['features']:
    polygon = shape(feature['geometry'])
    if polygon.contains(point):
        print( feature['properties']['area_numbe'])
        

43


In [ ]:
feature['properties']['area_numbe']

'9'

In [ ]:
def getpoint(address):
    loc = geolocator.geocode(address)
    latlong = [loc.latitude, loc.longitude]
    point = Point(latlong[1], latlong[0])   
    return point

In [ ]:
def findarea(point):    
    for feature in commarea['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
           return feature['properties']['area_numbe']
        else:
            continue
        

In [ ]:
def get_commarea_offenders(dataframe):
    dataframe['community_area']=''
    i=0
    for row in dataframe.index:
        try:
            point = getpoint(dataframe['fixed_addr'][row])
            dataframe['community_area'][row] = findarea(point)
        except:
            dataframe['community_area'][row] = 'no match to osm'
        print(f'{round((i/dataframe.shape[0]),4)*100}% done')
        i+=1
    return dataframe


In [ ]:
get_commarea_env(merged_envcomplaints)


C:\Users\tkkim\AppData\Local\Temp\ipykernel_1284\2635954889.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['community_area'][row] = feature['properties']['area_numbe']


In [ ]:
merged_crimes.to_csv(f'./results/crimes_cleaned_{output_name}.csv', index=False)
#merged_offenders.to_csv('./results/offeders_cleaned.csv', index=False)
merged_envcomplaints.to_csv(f'./results/crimes_envcomplaints_{output_name}.csv', index=False)

In [ ]:
#grouped_offenders_by_commarea = merged_offenders.groupby('community_area').agg(total_offenders_in_ca=('last','count')).reset_index()
#grouped_offenders_by_commarea.to_csv('./results/offenders_by_community_area.csv', index=False)
grouped_offenders_by_commarea = pd.read_csv('./results/offenders_by_community_area.csv')

In [ ]:
grouped_complaints_by_commarea = merged_envcomplaints.groupby('community_area').agg(total_complaints_in_ca=('complaint_id','count')).reset_index()
grouped_complaints_by_commarea.to_csv(f'./results/envcomplaints_by_community_area_{output_name}.csv', index=False)

In [ ]:
grouped_offenders_by_commarea['community_area'] = grouped_offenders_by_commarea['community_area'].replace('no match to osm', 999).astype(int)
grouped_offenders_by_commarea


,community_area,total_offenders_in_ca
0,1,21
1,10,15
2,11,14
3,12,6
4,13,3
...,...,...
71,76,11
72,77,24
73,8,16
74,9,4


In [ ]:
big_df = merged_crimes.merge(grouped_offenders_by_commarea, how='left', on='community_area')

In [ ]:
grouped_complaints_by_commarea = grouped_complaints_by_commarea[grouped_complaints_by_commarea['community_area'] != '']

In [ ]:
grouped_complaints_by_commarea['community_area'] = grouped_complaints_by_commarea['community_area'].astype('int64')

In [ ]:
big_df = big_df.merge(grouped_complaints_by_commarea, how='left', on='community_area')

In [ ]:
big_df.to_csv(f'./results/{output_name}.csv', index=False)

In [ ]:
big_df['date'].max()

Timestamp('2023-12-31 00:00:00')